# import library

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install wxee rasterio

In [ ]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio
import numpy as np
from datetime import datetime

# import constant

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

# import xarray sample

In [ ]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')
soil_wc

# GEE

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/w/private-key.json"
)

ee.Initialize(credentials)

# get landuse data as xarray

In [ ]:
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])

landuse_fao = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")



landuse_fao = landuse_fao.filterDate('2022-01-01','2022-12-31').filterBounds(shape)


landuse_fao = landuse_fao.set("system:time_start", ee.Date("2022-01-01"))
arr_landuse_fao = landuse_fao.wx.to_xarray(scale=10000, crs='EPSG:4326')


xds_repr_match_landuse_fao = arr_landuse_fao.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.mode)
xds_repr_match_landuse_fao


In [ ]:
xds_repr_match_landuse_fao.time

In [ ]:
new_times = pd.to_datetime('2022-01-01T00:00:00.000000000')

# old_time_index = xds_repr_match_landuse_fao.time.to_index().get_loc('2022-01-01T01:30:00.000000000')


# xds_repr_match_landuse_fao['time'][old_time_index] = new_times



# Create a new dataset with the desired time values
# new_dataset = xds_repr_match_landuse_fao.assign_coords(time = new_times)
new_dataset = xds_repr_match_landuse_fao.assign_coords(time=("time", new_times))

new_dataset


In [ ]:
import numpy as np
np.unique(xds_repr_match_landuse_fao["b1"])

# get class landuse file

In [ ]:
df_1 = pd.read_excel("/content/drive/MyDrive/w/landuse_fao.xlsx", sheet_name="Sheet1")
df_1

In [ ]:
column_titles = df_1.columns.tolist()
omitted_list = column_titles[1:]
column_titles

# map dataframe on landse data

In [ ]:
column_titles = df_1.columns.tolist()
omitted_list = column_titles[1:]

df = xds_repr_match_landuse_fao.to_dataframe()

for col in omitted_list:

    land_use_fao_dict = dict(zip(df_1['Class Value'], df_1[col]))

    df[col] = df['b1'].map(land_use_fao_dict)


landuse_name_da = df.to_xarray()
landuse_name_da = landuse_name_da.transpose('time', 'y', 'x')
landuse_name_da

In [ ]:
xds_repr_match_landuse_fao.to_netcdf("/content/drive/MyDrive/w/land_use_fao_2022001.nc")